In [35]:
!pip install streamlit pyngrok plotly pandas matplotlib


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import streamlit as st
from pyngrok import ngrok
from datetime import datetime


In [37]:
# Heart rate anomaly: <50 or >100 bpm
def detect_heart_rate_anomalies(df):
    threshold_high = 100
    threshold_low = 50
    df['hr_anomaly'] = df['heart_rate'].apply(lambda x: 1 if x>threshold_high or x<threshold_low else 0)
    return df

# Sleep anomaly: <4 or >12 hours
def detect_sleep_anomalies(df):
    df['sleep_anomaly'] = df['sleep_hours'].apply(lambda x: 1 if x<4 or x>12 else 0)
    return df

# Steps anomaly: <200 or >20000
def detect_steps_anomalies(df):
    df['steps_anomaly'] = df['steps'].apply(lambda x: 1 if x<200 or x>20000 else 0)
    return df


In [38]:
%%writefile dashboard.py
import streamlit as st
import pandas as pd
import plotly.express as px

# ---------- STYLES ----------
def add_bg():
    st.markdown("""
    <style>
        .main {
            background: linear-gradient(135deg,#fdfcfb,#e2d1c3);
        }
        h1 {
            color:#3d0066;
        }
        .stMetric label {
            font-size:16px !important;
        }
    </style>
    """, unsafe_allow_html=True)

add_bg()

# ---------- ANOMALY FUNCTIONS ----------
def detect_heart_rate_anomalies(df):
    threshold_high, threshold_low = 100, 50
    df['hr_anomaly'] = df['heart_rate'].apply(
        lambda x: 1 if x>threshold_high or x<threshold_low else 0)
    return df

def detect_sleep_anomalies(df):
    df['sleep_anomaly'] = df['sleep_hours'].apply(lambda x: 1 if x<4 or x>12 else 0)
    return df

def detect_steps_anomalies(df):
    df['steps_anomaly'] = df['steps'].apply(lambda x: 1 if x<200 or x>20000 else 0)
    return df

# ---------- UI ----------
st.title(" FitPulse Health Anomaly Dashboard ")

uploaded_file = st.file_uploader("📁 Upload your fitness data (CSV/JSON)", type=["csv","json"])

if uploaded_file:
    if uploaded_file.name.endswith(".csv"):
        df = pd.read_csv(uploaded_file)
    else:
        df = pd.read_json(uploaded_file)

    # Normalize column names (lowercase, strip spaces)
    df.columns = df.columns.str.lower().str.strip()

    # Map common variations to expected names
    rename_map = {
        'avg_heart_rate': 'heart_rate',
        'heartrate': 'heart_rate',
        'hr': 'heart_rate',
        'min_heart_rate': 'heart_rate',
        'max_heart_rate': 'heart_rate',
        'total_sleep_minutes': 'sleep_hours',
        'sleep': 'sleep_hours',
        'steps_count': 'steps',
        'step_count': 'steps'
    }
    df = df.rename(columns=rename_map)
    # Remove duplicate columns
    df = df.loc[:, ~df.columns.duplicated()]
    if 'steps' not in df.columns:
        if 'totalsteps' in df.columns:
            df['steps'] = df['totalsteps']
        else:
            df['steps'] = 0

# If multiple heart rate fields exist → choose average
    if 'avg_heart_rate' in df.columns and 'heart_rate' not in df.columns:
       df.rename(columns={'avg_heart_rate': 'heart_rate'}, inplace=True)

# If min/max HR present, drop them (we don't need them for anomaly)
    for col in ['min_heart_rate', 'max_heart_rate']:
      if col in df.columns:
        df.drop(columns=[col], inplace=True)


    # convert minutes → hours if needed
    if 'sleep_hours' not in df.columns and 'total_sleep_minutes' in df.columns:
        df['sleep_hours'] = df['total_sleep_minutes'] / 60

    # add date if missing
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
    else:
        df['date'] = pd.date_range(start='2024-01-01', periods=len(df))

    st.success("File uploaded successfully!")
    st.subheader(" Raw Data Preview")
    st.dataframe(df.head())


    # Apply anomaly detection
    df = detect_heart_rate_anomalies(df)
    df = detect_sleep_anomalies(df)
    df = detect_steps_anomalies(df)

    # ---- METRICS ----
    st.subheader(" Daily Health Summary")
    c1, c2, c3 = st.columns(3)
    c1.metric(" HR Anomalies", int(df['hr_anomaly'].sum()))
    c2.metric(" Sleep Anomalies", int(df['sleep_anomaly'].sum()))
    c3.metric(" Step Anomalies", int(df['steps_anomaly'].sum()))

    # ---- HEART RATE ----
    st.markdown("###  Heart Rate Trends")
    fig_hr = px.line(df, x='date', y='heart_rate', color='hr_anomaly',
                     title="Heart Rate Trends (Red = anomaly)")
    st.plotly_chart(fig_hr)

    # ---- SLEEP ----
    st.markdown("###  Sleep Duration Patterns")
    fig_sleep = px.line(df, x='date', y='sleep_hours', color='sleep_anomaly',
                        title="Sleep Duration (Blue = anomaly)")
    st.plotly_chart(fig_sleep)

    # ---- STEPS ----
    st.markdown("###  Step Count Behavior")
    fig_steps = px.line(df, x='date', y='steps', color='steps_anomaly',
                        title="Activity Levels (Green = anomaly)")
    st.plotly_chart(fig_steps)

    # ---- DOWNLOAD ----
    st.info(" Click below to download anomaly report")
    df_report = df[['date','heart_rate','hr_anomaly','sleep_hours','sleep_anomaly','steps','steps_anomaly']]
    csv = df_report.to_csv(index=False)
    st.download_button(label="Download CSV", data=csv,
                       file_name="anomaly_report.csv", mime="text/csv")


Overwriting dashboard.py


In [48]:
import os

# Run Streamlit app in the background
!streamlit run dashboard.py &>/dev/null&

# Terminate ngrok tunnels if any are running
# This is important for re-running the cell after an error or a change in authentication
if os.path.exists(".ngrok2"): # Check if ngrok config file exists
    try:
        ngrok.kill()
    except:
        pass

# Authenticate ngrok (replace 'YOUR_NGROK_AUTH_TOKEN' with your actual token)
# You can get your auth token from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("38D0HjtRqkYF5QIP4wCJb7tyqv7_6ZYM4o8rnJSJbpiCkhsMP")

# Create public URL with ngrok
public_url = ngrok.connect('8501')
public_url

<NgrokTunnel: "https://disproportionable-overforged-rank.ngrok-free.dev" -> "http://localhost:8501">

In [47]:
from pyngrok import ngrok

ngrok.kill()
print("All ngrok processes terminated.")

All ngrok processes terminated.
